# SageMaker Edge Manager Example

1. [Introduction](#Introduction)
2. [Demo Setup](#Demo-Setup)
    1. [Launch EC2 Instance](#Launch-EC2-Instance)
3. [Compile Model using SageMaker Neo](#Compile-Model-using-SageMaker-Neo)
    1. [Load pretrained model](#Load-pretrained-model)
6. [Deploy Model using Sagemaker Edge Manager](#Deploy-Model-using-Sagemaker-Edge-Manager)
    1. [Package Model](#Package-Model)
    2. [Create AWS IoT thing](#Create-AWS-IoT-thing)
    3. [Create Device Fleet](#Create-Device-Fleet)
    4. [Create and register client certificate with AWS IoT](#Create-and-register-client-certificate-with-AWS-IoT)
7. [Inference on Edge](#Inference-on-Edge)
    1. [Setup Sagemaker Edge Manager Agent](#Setup-Sagemaker-Edge-Manager-Agent) 
    2. [Load Model](#Load-Model)
    3. [List Models](#List-Models)
    4. [Run Predict](#Run-Predict)
    5. [Capture Data](#Capture-Data)
    6. [Unload Model](#Unload-Model)
8. [Clean Up](#Clean-Up)
9. [Appendix](#Appendix)
    1. [(Optional)Install CloudWatch Agent](#(Optional)Install-CloudWatch-Agent )

## Introduction

SageMaker Edge Manager is a service from Amazon SageMaker that lets you:

+ prepares custom models for edge device hardware
+ includes a runtime for running machine learning inference efficiently on edge devices
+ enables the device to send samples of data from each model securely to SageMaker for relabeling and retraining.

There are two main components to this service:
+ SageMaker Edge Manager in the Cloud 
+ SageMaker Edge Agent on the Edge device

This nootebook demonstrates the end-to-end workflow for getting a running Sagemaker Edge on the edge device. This will involve the following steps:

+ Compile the model using SageMaker Neo
+ Package the compiled model with Sagemaker Edge Manager
+ Deploy with Sagemaker Edge Manager Agent
+ Run inference with the model
+ Capture model's input and output data to S3

**Note**:
Typically, the SageMaker Edge Agent is run on an Edge device. For the sake of this notebook, we will run the Agent on an EC2 instance. We show how to package the compiled model and then load it to the Agent on the Edge Device to make predictions with. Finally, we show how to capture model's input and output to S3 via the Agent.

This notebook is intented only for notebook instances. When you run this notebook, choose the kernel: `conda_tensorflow_p36`

**Please note**: There are pricing implications to the use of this notebook. Please refer to [Edge Manager](https://aws.amazon.com/sagemaker/edge-manager/pricing) for more information.

## Demo Setup

We need an AWS account role with SageMaker access. This role is used to give SageMaker access to S3, launch an EC2 instance and create components and deployments in Greengrass.

In [1]:
import sagemaker
from sagemaker import get_execution_role
import boto3
import botocore
import json

role = get_execution_role()
sess = sagemaker.Session()
region = boto3.Session().region_name

In [2]:
print(role)

arn:aws:iam::699391019698:role/Admin


Locate the above printed sagemaker role from [IAM console](https://console.aws.amazon.com/iam), find and attach the following policies to role:

- AmazonEC2FullAccess 
- AmazonEC2RoleforSSM 
- AmazonSSMManagedInstanceCore 
- AmazonSSMFullAccess 
- AWSGreengrassFullAccess
- AWSIoTFullAccess 

You can find more information about how to attach policies to role [here](https://docs.aws.amazon.com/IAM/latest/UserGuide/access_policies_manage-attach-detach.html#add-policies-console).

**If you try this example with a real device, only attach AWSIoTFullAccess to create certificates on AWS IoT.**

We then need an S3 bucket that would be used for storing the model artifacts generated after compilation and packaged artifacts generated after edge packaging job.

In [3]:
# S3 bucket and folders for saving model artifacts.
# Feel free to specify different bucket/folders here if you wish.
bucket = sess.default_bucket()
folder = "DEMO-Sagemaker-Edge"
compilation_output_sub_folder = folder + "/compilation-output"
iot_folder = folder + "/iot"

# S3 Location to save the model artifact after compilation
s3_compilation_output_location = "s3://{}/{}".format(bucket, compilation_output_sub_folder)

Finally we upload the test image to S3 bucket. This image will be used in inference later.

In [4]:
darknet_img_path = sess.upload_data("darknet.bmp", bucket, iot_folder)
keras_img_path = sess.upload_data("keras.bmp", bucket, iot_folder)

### Launch EC2 Instance

As mentioned earlier, this EC2 instance is used in place of an Edge device for running the agent software.

In [5]:
ec2_client = boto3.client("ec2", region_name=region)

Generate key pair for EC2 instance, save the key pem file. We can use this key with SSH to connect to the instance. But in this notebook example, we will not use SSH, instead, we will use AWS Systems Manager to send commands to the instance.

In [ ]:
key_pairs = ec2_client.describe_key_pairs()
key_names = list(map(lambda x: x["KeyName"], key_pairs["KeyPairs"]))

key_name = "ec2-key-pair"

if key_name in key_names:
    ec2_key_pair = ec2_client.delete_key_pair(
        KeyName=key_name,
    )

In [ ]:
ec2_key_pair = ec2_client.create_key_pair(
    KeyName=key_name,
)

key_pair = str(ec2_key_pair["KeyMaterial"])
key_pair_file = open("ec2-key-pair.pem", "w")
key_pair_file.write(key_pair)
key_pair_file.close()

Create a role for the EC2 instance we are going to use. Read for detailed information about [IAM roles for Amazon EC2](https://docs.aws.amazon.com/AWSEC2/latest/UserGuide/iam-roles-for-amazon-ec2.html).

Following steps here to [create an IAM role](https://docs.aws.amazon.com/AWSEC2/latest/UserGuide/iam-roles-for-amazon-ec2.html#create-iam-role). Note down the role name and role ARN, role name will be used when we launch the EC2 instance, and role ARN will be needed to create inline policy.

After creation, make sure the following policies are attached to role:

- AmazonS3FullAccess 
- AmazonSSMManagedInstanceCore 
- CloudWatchAgentAdminPolicy 


Locate the same sagemaker role used for this notebook in [Demo Setup](#Demo-Setup) in [IAM console](https://console.aws.amazon.com/iam), click `Add inline policy` button on the role summary page, choose JSON format and replace the content with below statement:

Before copy the following content, make sure you use the EC2 role ARN you just created in the `Resource` field for `iam:PassRole` action.

```
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": "iam:PassRole",
            "Resource": "arn:aws:iam::<account>:role/<role-name>"
        },
        {
            "Effect": "Allow",
            "Action": [
                "iot:AddThingToThingGroup",
                "iot:AttachPolicy",
                "iot:AttachThingPrincipal",
                "iot:CreateKeysAndCertificate",
                "iot:CreatePolicy",
                "iot:CreateRoleAlias",
                "iot:CreateThing",
                "iot:CreateThingGroup",
                "iot:DescribeEndpoint",
                "iot:DescribeRoleAlias",
                "iot:DescribeThingGroup",
                "iot:GetPolicy",
                "iam:GetRole",
                "iam:CreateRole",
                "iam:PassRole",
                "iam:CreatePolicy",
                "iam:AttachRolePolicy",
                "iam:GetPolicy",
                "sts:GetCallerIdentity"
            ],
            "Resource": "*"
        },
        {
            "Sid": "DeployDevTools",
            "Effect": "Allow",
            "Action": [
                "greengrass:CreateDeployment",
                "iot:CancelJob",
                "iot:CreateJob",
                "iot:DeleteThingShadow",
                "iot:DescribeJob",
                "iot:DescribeThing",
                "iot:DescribeThingGroup",
                "iot:GetThingShadow",
                "iot:UpdateJob",
                "iot:UpdateThingShadow"
            ],
            "Resource": "*"
        }
    ]
}
```

Launch an EC2 C5 instance. In this example we will use aws deep learning ami.

In [6]:
ami_map = {
    "us-east-1": "ami-063585f0e06d22308",
    "us-east-2": "ami-01bd6a1621a6968d7",
    "us-west-2": "ami-0bc87a16c757a7f07",
    "eu-central-1": "ami-01227276a4e5a4a31",
    "ap-northeast-1": "ami-03b8cfea5460e4881",
    "eu-west-1": "ami-006ff58f5247c50eb",
}

In [10]:
ec2_profile_name = "SMEdgeManageExampleRole"  # the name of the role created for EC2

ec2_instance = ec2_client.run_instances(
     ImageId=ami_map[region],
     MinCount=1,
     MaxCount=1,
     InstanceType='c5.large',
     KeyName='ec2',
     IamInstanceProfile={
        'Name': ec2_profile_name}
)

In [11]:
instance_id = ec2_instance["Instances"][0]["InstanceId"]  # will used for running inference later

## Compile Model using SageMaker Neo

Create Sagemaker client.

In [38]:
sagemaker_client = boto3.client("sagemaker", region_name=region)

### Download pretrained darknet model

In [16]:
!wget -O yolov3-tiny.cfg "https://github.com/pjreddie/darknet/blob/master/cfg/yolov3-tiny.cfg?raw=true"
!wget https://pjreddie.com/media/files/yolov3-tiny.weights

--2021-06-23 16:11:00--  https://github.com/pjreddie/darknet/blob/master/cfg/yolov3-tiny.cfg?raw=true
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/pjreddie/darknet/raw/master/cfg/yolov3-tiny.cfg [following]
--2021-06-23 16:11:00--  https://github.com/pjreddie/darknet/raw/master/cfg/yolov3-tiny.cfg
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3-tiny.cfg [following]
--2021-06-23 16:11:01--  https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3-tiny.cfg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awa

In [17]:
import tarfile

with tarfile.open("yolov3-tiny.tar.gz", mode="w:gz") as archive:
    archive.add("yolov3-tiny.cfg")
    archive.add("yolov3-tiny.weights")

In [18]:
darknet_model_path = sess.upload_data("yolov3-tiny.tar.gz", bucket, folder)

**Note**: When calling ``create_compilation_job()`` user is expected to provide all the correct input shapes required by the model for successful compilation. If we are using a different model, we need to specify the framework and data shape correctly..

In [19]:
darknet_model_data_shape = '{"data":[1,3,416,416]}'
darknet_model_framework = "darknet"
target_device = "ml_c5"

In [23]:
import time
role = "arn:aws:iam::699391019698:role/service-role/AmazonSageMaker-ExecutionRole-20190424T105861"
darknet_compilation_job_name = "Sagemaker-Edge-" + str(time.time()).split(".")[0]
print("Compilation job for %s started" % darknet_compilation_job_name)

response = sagemaker_client.create_compilation_job(
    CompilationJobName=darknet_compilation_job_name,
    RoleArn=role,
    InputConfig={
        "S3Uri": darknet_model_path,
        "DataInputConfig": darknet_model_data_shape,
        "Framework": darknet_model_framework.upper(),
    },
    OutputConfig={
        "S3OutputLocation": s3_compilation_output_location,
        "TargetDevice": target_device,
    },
    StoppingCondition={"MaxRuntimeInSeconds": 900},
)

print(response)

# Poll every 30 sec
while True:
    response = sagemaker_client.describe_compilation_job(
        CompilationJobName=darknet_compilation_job_name
    )
    if response["CompilationJobStatus"] == "COMPLETED":
        break
    elif response["CompilationJobStatus"] == "FAILED":
        print(response)
        raise RuntimeError("Compilation failed")
    print("Compiling ...")
    time.sleep(10)
print("Done!")

Compilation job for Sagemaker-Edge-1624459072 started
{'CompilationJobArn': 'arn:aws:sagemaker:eu-west-1:699391019698:compilation-job/Sagemaker-Edge-1624459072', 'ResponseMetadata': {'RequestId': '33f964ae-d423-4175-bfba-bf506b2babd0', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '33f964ae-d423-4175-bfba-bf506b2babd0', 'content-type': 'application/x-amz-json-1.1', 'content-length': '106', 'date': 'Wed, 23 Jun 2021 14:37:52 GMT'}, 'RetryAttempts': 0}}
Compiling ...
Compiling ...
Compiling ...
Compiling ...
Compiling ...
Compiling ...
Compiling ...
Compiling ...
Compiling ...
Compiling ...
Compiling ...
Compiling ...
Compiling ...
Compiling ...
Compiling ...
Compiling ...
Compiling ...
Compiling ...
Compiling ...
Done!


## Package Model using Sagemaker Edge Manager

In this section, we will walk through packaging two models that achieve different goals. One is an Image Classification model (from Keras framework) and another is an Object Detection Model from DarkNet framework. This showcases the versatility of SageMaker Edge Manager.

### Package Darknet Model

Before we can deploy the compiled model to edge devices, we need to package the model with Sagemaker Edge Manager cloud service.

In [67]:
darknet_packaged_model_name = "darknet-model"
darknet_model_version = "1.0.0"
darknet_component_name = "com.model.darknet"
darknet_model_package = "{}-{}.tar.gz".format(darknet_packaged_model_name, darknet_model_version)

In [70]:
import json
darknet_packaging_job_name = darknet_compilation_job_name + "-packaging"
response = sagemaker_client.create_edge_packaging_job(
    RoleArn=role,
    OutputConfig={
        "PresetDeploymentType": "GreengrassV2Component",
        "PresetDeploymentConfig": json.dumps({"ComponentName":darknet_component_name, "ComponentVersion":darknet_model_version}),
        "S3OutputLocation": s3_compilation_output_location,
    },
    ModelName=darknet_packaged_model_name,
    ModelVersion=darknet_model_version,
    EdgePackagingJobName=darknet_packaging_job_name,
    CompilationJobName=darknet_compilation_job_name,
)

print(response)

# Poll every 30 sec
while True:
    job_status = sagemaker_client.describe_edge_packaging_job(
        EdgePackagingJobName=darknet_packaging_job_name
    )
    if job_status["EdgePackagingJobStatus"] == "COMPLETED":
        break
    elif job_status["EdgePackagingJobStatus"] == "FAILED":
        raise RuntimeError("Edge Packaging failed")
    print("Packaging ...")
    time.sleep(30)
print("Done!")

{'ResponseMetadata': {'RequestId': '7f076c23-e040-4a8b-8b2f-233dd131870f', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '7f076c23-e040-4a8b-8b2f-233dd131870f', 'content-type': 'application/x-amz-json-1.1', 'content-length': '0', 'date': 'Wed, 23 Jun 2021 15:29:58 GMT'}, 'RetryAttempts': 0}}
Packaging ...
Done!


In [44]:
darknet_model_data = job_status["ModelArtifact"]

### Download pretrained Keras model

In [45]:
import tensorflow as tf

model = tf.keras.applications.MobileNetV2()
model.save("mobilenet_v2.h5")

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
import tarfile

with tarfile.open("mobilenet_v2.tar.gz", mode="w:gz") as archive:
    archive.add("mobilenet_v2.h5")

In [ ]:
keras_model_path = sess.upload_data("mobilenet_v2.tar.gz", bucket, folder)

**Note**: When calling ``create_compilation_job()`` user is expected to provide all the correct input shapes required by the model for successful compilation. If we are using a different model, we need to specify the framework and data shape correctly..

In [ ]:
keras_model_data_shape = '{"input_1":[1,3,224,224]}'
keras_model_framework = "keras"
target_device = "ml_c5"

In [ ]:
import time

keras_compilation_job_name = "Sagemaker-Edge-" + str(time.time()).split(".")[0]
print("Compilation job for %s started" % keras_compilation_job_name)

response = sagemaker_client.create_compilation_job(
    CompilationJobName=keras_compilation_job_name,
    RoleArn=role,
    InputConfig={
        "S3Uri": keras_model_path,
        "DataInputConfig": keras_model_data_shape,
        "Framework": keras_model_framework.upper(),
    },
    OutputConfig={
        "S3OutputLocation": s3_compilation_output_location,
        "TargetDevice": target_device,
    },
    StoppingCondition={"MaxRuntimeInSeconds": 900},
)

print(response)

# Poll every 30 sec
while True:
    response = sagemaker_client.describe_compilation_job(
        CompilationJobName=keras_compilation_job_name
    )
    if response["CompilationJobStatus"] == "COMPLETED":
        break
    elif response["CompilationJobStatus"] == "FAILED":
        raise RuntimeError("Compilation failed")
    print("Compiling ...")
    time.sleep(30)
print("Done!")

### Package Keras Model

In [ ]:
keras_packaged_model_name = "keras-model"
keras_model_version = "1.0.0"
keras_component_name = "com.model.keras"
keras_model_package = "{}-{}.tar.gz".format(keras_packaged_model_name, keras_model_version)

In [ ]:
keras_packaging_job_name = keras_compilation_job_name + "-packaging"
response = sagemaker_client.create_edge_packaging_job(
    RoleArn=role,
    OutputConfig={
        "PresetDeploymentType": "GreengrassV2Component",
        "PresetDeploymentConfig": json.dumps({"ComponentName":keras_component_name, "ComponentVersion":keras_model_version}),
        "S3OutputLocation": s3_compilation_output_location,
    },
    ModelName=keras_packaged_model_name,
    ModelVersion=keras_model_version,
    EdgePackagingJobName=keras_packaging_job_name,
    CompilationJobName=keras_compilation_job_name,
)

print(response)

# Poll every 30 sec
while True:
    job_status = sagemaker_client.describe_edge_packaging_job(
        EdgePackagingJobName=keras_packaging_job_name
    )
    if job_status["EdgePackagingJobStatus"] == "COMPLETED":
        break
    elif job_status["EdgePackagingJobStatus"] == "FAILED":
        raise RuntimeError("Edge Packaging failed")
    print("Packaging ...")
    time.sleep(30)
print("Done!")

In [ ]:
keras_model_data = job_status["ModelArtifact"]


### Install Greengrass

SageMaker Edge Manager can use AWS IoT Greengrass to deploy the agent, the model and the inference application to the edge device.

AWS IoT Greengrass provides all the necessary features to manage applications on remote devices in a secure and scalable way. To learn more about Greengrass, head to the [documentation](https://docs.aws.amazon.com/greengrass/v2/developerguide/what-is-iot-greengrass.html). 

The SageMaker Edge Manager agent leverages the AWS credentials provided by the [Token exchange service(https://docs.aws.amazon.com/greengrass/v2/developerguide/token-exchange-service-component.html) component to securely communicate with the SageMaker Edge Manager backend.


In [50]:
ssm_client = boto3.client("ssm", region_name=region)

With the following command we are installing Greengrass on the EC2 instance. If you are using a real device, connect to the device via SSH, ensure that you have both Java v8 or above and the Unzip command and then run the following commands (replace `<your_region>` with the correct value). To run this command on the device you also need IAM credentials with at least the permissions listed [here](https://docs.aws.amazon.com/greengrass/v2/developerguide/provision-minimal-iam-policy.html).

```bash
curl -s https://d2s8p88vqu9w66.cloudfront.net/releases/greengrass-nucleus-latest.zip > greengrass-nucleus-latest.zip \
               && unzip greengrass-nucleus-latest.zip -d GreengrassCore,
sudo -E java -Droot="/greengrass/v2" -Dlog.store=FILE -jar ./GreengrassCore/lib/Greengrass.jar \
    --aws-region <your_region> \
    --thing-name GreengrassSMEdgeManagerDevice -trn SageMakerTESRole -tra SageMakerTESRoleAlias \ 
    --thing-group-name GreengrassSMEdgeManagerGroup \ 
    --component-default-user ggc_user:ggc_group --provision true --setup-system-service true --deploy-dev-tools true
```

In [56]:
response = ssm_client.send_command(
    InstanceIds=[instance_id],
    DocumentName="AWS-RunShellScript",
    OutputS3BucketName=bucket,
    OutputS3KeyPrefix=folder,
    Parameters={
        "commands": [
            "#!/bin/bash",
            "curl -s https://d2s8p88vqu9w66.cloudfront.net/releases/greengrass-nucleus-latest.zip > greengrass-nucleus-latest.zip && unzip greengrass-nucleus-latest.zip -d GreengrassCore",
            'sudo -E java -Droot="/greengrass/v2" -Dlog.store=FILE -jar ./GreengrassCore/lib/Greengrass.jar --aws-region eu-west-1 --thing-name GreengrassSMEdgeManagerDevice -trn SageMakerTESRole -tra SageMakerTESRoleAlias --thing-group-name GreengrassSMEdgeManagerGroup --component-default-user ggc_user:ggc_group --provision true --setup-system-service true --deploy-dev-tools true'
        ]
    }
)

In [61]:
ssm_client.get_command_invocation(
    CommandId=response["Command"]["CommandId"],
    InstanceId=instance_id,
)

{'CommandId': 'aeb69c81-c1a6-4d87-b87c-e8be50ad092a',
 'InstanceId': 'i-086550044d97bdfaf',
 'Comment': '',
 'DocumentName': 'AWS-RunShellScript',
 'DocumentVersion': '$DEFAULT',
 'PluginName': 'aws:runShellScript',
 'ResponseCode': -1,
 'ExecutionEndDateTime': '',
 'Status': 'InProgress',
 'StatusDetails': 'InProgress',
 'StandardOutputContent': '',
 'StandardOutputUrl': '',
 'StandardErrorContent': '',
 'StandardErrorUrl': '',
 'CloudWatchOutputConfig': {'CloudWatchLogGroupName': '',
  'CloudWatchOutputEnabled': False},
 'ResponseMetadata': {'RequestId': 'e076867a-be94-44ba-b175-1f983e270680',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Wed, 23 Jun 2021 15:07:45 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '479',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'e076867a-be94-44ba-b175-1f983e270680'},
  'RetryAttempts': 0}}

### Create Device Fleet

#### Modify the IAM role for device fleet

Configure an IAM role in your AWS account that will be assumed by the credentials provider on behalf of the devices in your device fleet. 


Go to [IAM console](https://console.aws.amazon.com/iam/home?#/roles/SageMakerTESRole), and look for the role create role for IoT, attach the following policies:

- AmazonSageMakerEdgeDeviceFleetPolicy

Edit then the [trust relationship](https://console.aws.amazon.com/iam/home?#/roles/SageMakerTESRole?section=trust) as follow:
```
{
  "Version": "2012-10-17",
  "Statement": [
      {
        "Effect": "Allow",
        "Principal": {"Service": "credentials.iot.amazonaws.com"},
        "Action": "sts:AssumeRole"
      },
      {
        "Effect": "Allow",
        "Principal": {"Service": "sagemaker.amazonaws.com"},
        "Action": "sts:AssumeRole"
      }
  ]
}
```

Note down the role ARN, it will be later used for creating the device fleet.

In [62]:
role_arn="<your role arn>"

In [64]:
device_fleet_name ="demo-device-fleet" + str(time.time()).split('.')[0]

sagemaker_client.create_device_fleet(
    DeviceFleetName=device_fleet_name,
    RoleArn=role_arn,
    OutputConfig={
        'S3OutputLocation': s3_compilation_output_location
    }
)

{'ResponseMetadata': {'RequestId': 'ce0c9193-3a31-47bb-bd70-c2a9775db3dd',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'ce0c9193-3a31-47bb-bd70-c2a9775db3dd',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Wed, 23 Jun 2021 15:14:11 GMT'},
  'RetryAttempts': 0}}

#### Register device to the fleet

In [66]:
device_name = "GreengrassSMEdgeManagerDevice"

sagemaker_client.register_devices(
    DeviceFleetName=device_fleet_name,
    Devices=[
        {
            "DeviceName": device_name,
            "IotThingName": device_name,
            "Description": "this is a sample virtual device",
        }
    ],
)

{'ResponseMetadata': {'RequestId': 'aa668711-74f2-4002-a4a8-c7bcb624667f',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'aa668711-74f2-4002-a4a8-c7bcb624667f',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Wed, 23 Jun 2021 15:15:26 GMT'},
  'RetryAttempts': 0}}

## Inference on Edge

In this example, we will use [AWS Iot Greengrass](https://docs.aws.amazon.com/) to remotely deploy the agent, the model and the inference application.

The [SageMaker Edge Manager component](https://docs.aws.amazon.com/greengrass/v2/developerguide/sagemaker-edge-manager-component.html) is already provided an will be used to deploy and run the agent on the device.

The model component has been created for you by the packaging job. You should have 2 models: `com.model.keras` and `com.model.darknet`

In order to be able to use the model, we also need an application component to load the model and invoke it. In the next section we are going to see how to create such component.

## Create the inference application component

We will use a python application to 

### Load Model

In this section, we show the model management capabilities offered by SageMaker Edge Manager. We will load the two compiled and packaged models with the SageMaker Edge Agent. This keeps both models ready to run inference with. As you will see, once the models are loaded you can run multiple inferences as many times as necessary until the models are unloaded. This relieves the client applications from the logic and operational burden of managing them separately. These models are now simply an API away from running inference with.

When loading the model with SageMaker Edge Agent, the argument to the API points the Agent to a directory containing the packaged model (without any extraneous files within the directory). 

#### Load darknet model

`darknet_model` is the path containing the packaged model in this notebook. `demo-darknet` is the name given to this model. This name will be used later to refer to this model for, making predictions, capturing data, unload.

In [ ]:
load_darknet_model_out = ssm_client.send_command(
    InstanceIds=[instance_id],
    DocumentName="AWS-RunShellScript",
    OutputS3BucketName=bucket,
    OutputS3KeyPrefix=folder,
    Parameters={
        "commands": [
            "cd /demo",
            "./bin/sagemaker_edge_agent_client_example LoadModel darknet_model demo-darknet",
        ]
    },
)

In [ ]:
ssm_client.get_command_invocation(
    CommandId=load_darknet_model_out["Command"]["CommandId"],
    InstanceId=instance_id,
)

#### Load keras model

`keras_model` is the path containing the packaged model in this notebook. `demo-keras` is the name given to this model. This name will be used later to refer to this model for, making predictions, capturing data, unload.

In [ ]:
load_keras_model_out = ssm_client.send_command(
    InstanceIds=[instance_id],
    DocumentName="AWS-RunShellScript",
    OutputS3BucketName=bucket,
    OutputS3KeyPrefix=folder,
    Parameters={
        "commands": [
            "cd /demo",
            "./bin/sagemaker_edge_agent_client_example LoadModel keras_model demo-keras",
        ]
    },
)

In [ ]:
ssm_client.get_command_invocation(
    CommandId=load_keras_model_out["Command"]["CommandId"],
    InstanceId=instance_id,
)

### List Models

This API simply lists all the models and their names that are loaded with SageMaker Edge Agent. Note that the names shown here are same as the ones provided during the LoadModel in the previous sections.

In [ ]:
list_model_out = ssm_client.send_command(
    InstanceIds=[instance_id],
    DocumentName="AWS-RunShellScript",
    OutputS3BucketName=bucket,
    OutputS3KeyPrefix=folder,
    Parameters={"commands": ["cd /demo", "./bin/sagemaker_edge_agent_client_example ListModels"]},
)

In [ ]:
ssm_client.get_command_invocation(
    CommandId=list_model_out["Command"]["CommandId"],
    InstanceId=instance_id,
)

### Run Predict

In this API, we pass the model name, input data file that will be directly fed into the neural network, input tensor name that was passed earlier during the compilation phase, along with it's size and shape.

#### Run prediction on darknet model

In [ ]:
darknet_predict_out = ssm_client.send_command(
    InstanceIds=[instance_id],
    DocumentName="AWS-RunShellScript",
    OutputS3BucketName=bucket,
    OutputS3KeyPrefix=folder,
    Parameters={
        "commands": [
            "cd /demo",
            "./bin/sagemaker_edge_agent_client_example Predict demo-darknet darknet.bmp data 416 416 3",
        ]
    },
)

In [ ]:
ssm_client.get_command_invocation(
    CommandId=darknet_predict_out["Command"]["CommandId"],
    InstanceId=instance_id,
)

#### Run prediction on keras model

In [ ]:
keras_predict_out = ssm_client.send_command(
    InstanceIds=[instance_id],
    DocumentName="AWS-RunShellScript",
    OutputS3BucketName=bucket,
    OutputS3KeyPrefix=folder,
    Parameters={
        "commands": [
            "cd /demo",
            "./bin/sagemaker_edge_agent_client_example Predict demo-keras keras.bmp input_1 224 224 3",
        ]
    },
)

In [ ]:
ssm_client.get_command_invocation(
    CommandId=keras_predict_out["Command"]["CommandId"],
    InstanceId=instance_id,
)

### Capture Data

Capture the inputs and outputs of an inference call to cloud or disk. The specific parameters were configured earlier in the config file. 

In [ ]:
darknet_capture_out = ssm_client.send_command(
    InstanceIds=[instance_id],
    DocumentName="AWS-RunShellScript",
    OutputS3BucketName=bucket,
    OutputS3KeyPrefix=folder,
    Parameters={
        "commands": [
            "cd /demo",
            "./bin/sagemaker_edge_agent_client_example PredictAndCapture demo-darknet darknet.bmp data 416 416 3",
        ]
    },
)

In [ ]:
ssm_client.get_command_invocation(
    CommandId=darknet_capture_out["Command"]["CommandId"],
    InstanceId=instance_id,
)

In [ ]:
keras_capture_out = ssm_client.send_command(
    InstanceIds=[instance_id],
    DocumentName="AWS-RunShellScript",
    OutputS3BucketName=bucket,
    OutputS3KeyPrefix=folder,
    Parameters={
        "commands": [
            "cd /demo",
            "./bin/sagemaker_edge_agent_client_example PredictAndCapture demo-keras keras.bmp input_1 224 224 3",
        ]
    },
)

In [ ]:
ssm_client.get_command_invocation(
    CommandId=keras_capture_out["Command"]["CommandId"],
    InstanceId=instance_id,
)

### Unload Model

After unloading a model, the same name can be reused for future `LoadModel` APIs calls.

In [ ]:
unload_model_out = ssm_client.send_command(
    InstanceIds=[instance_id],
    DocumentName="AWS-RunShellScript",
    OutputS3BucketName=bucket,
    OutputS3KeyPrefix=folder,
    Parameters={
        "commands": [
            "cd /demo",
            "./bin/sagemaker_edge_agent_client_example UnloadModel demo-darknet",
            "./bin/sagemaker_edge_agent_client_example UnloadModel demo-keras",
        ]
    },
)

In [ ]:
ssm_client.get_command_invocation(
    CommandId=unload_model_out["Command"]["CommandId"],
    InstanceId=instance_id,
)

## Clean Up

Stop the Agent

In [ ]:
ssm_client.cancel_command(CommandId=agent_out["Command"]["CommandId"], InstanceIds=[instance_id])

Stop the EC2 instance

In [ ]:
ec2_client.stop_instances(InstanceIds=[instance_id])

Detach and delete policy

In [ ]:
iot_client.detach_policy(policyName=policy_name, target=iot_cert["certificateArn"])

iot_client.delete_policy(policyName=policy_name)

Deregister device and delete device fleet

In [ ]:
sagemaker_client.deregister_devices(DeviceFleetName=device_fleet_name, DeviceNames=[device_name])

sagemaker_client.delete_device_fleet(DeviceFleetName=device_fleet_name)

## Appendix

### (Optional)Install CloudWatch Agent 

In [ ]:
CW_log_config = {
    "agent": {
        "metrics_collection_interval": 10,
        "logfile": "/opt/aws/amazon-cloudwatch-agent/logs/amazon-cloudwatch-agent.log",
    },
    "logs": {
        "logs_collected": {
            "files": {
                "collect_list": [
                    {
                        "file_path": "/opt/aws/amazon-cloudwatch-agent/logs/amazon-cloudwatch-agent.log",
                        "log_group_name": "amazon-cloudwatch-agent.log",
                        "log_stream_name": "amazon-cloudwatch-agent.log",
                        "timezone": "UTC",
                    },
                    {
                        "file_path": "/opt/aws/amazon-cloudwatch-agent/logs/test.log",
                        "log_group_name": "test.log",
                        "log_stream_name": "test.log",
                        "timezone": "Local",
                    },
                ]
            }
        },
        "log_stream_name": "my_log_stream_name",
        "force_flush_interval": 15,
    },
}

In [ ]:
CW_file = open("cloudwatch.json", "w")
json.dump(CW_log_config, CW_file, indent=6)
CW_file.close()

In [ ]:
CW_config_path = sess.upload_data("cloudwatch.json", bucket, iot_folder)

In [ ]:
ssm_client.send_command(
    InstanceIds=[instance_id],
    DocumentName="AWS-RunShellScript",
    OutputS3BucketName=bucket,
    OutputS3KeyPrefix=folder,
    Parameters={
        "commands": [
            "#!/bin/bash",
            "aws s3 cp "
            + CW_config_path
            + " /opt/aws/amazon-cloudwatch-agent/etc/amazon-cloudwatch-agent.json",
        ]
    },
)

In [ ]:
ssm_client.send_command(
    InstanceIds=[instance_id],
    DocumentName="AWS-RunShellScript",
    OutputS3BucketName=bucket,
    OutputS3KeyPrefix=folder,
    Parameters={
        "commands": [
            "#!/bin/bash",
            "wget https://s3.amazonaws.com/amazoncloudwatch-agent/ubuntu/amd64/latest/amazon-cloudwatch-agent.deb",
            "sudo dpkg -i -E ./amazon-cloudwatch-agent.deb",
        ]
    },
)

Install Cloud Watch Agent to SSM agent.

In [ ]:
ssm_client.send_command(
    DocumentName="AWS-ConfigureAWSPackage",
    DocumentVersion="1",
    OutputS3BucketName=bucket,
    OutputS3KeyPrefix=folder,
    Targets=[
        {"Key": "InstanceIds", "Values": [instance_id]},
    ],
    TimeoutSeconds=600,
    Parameters={"action": ["Install"], "name": ["AmazonCloudWatchAgent"]},
    MaxConcurrency="50",
    MaxErrors="0",
)

To debug with CloudWatch, add a paramater `CloudWatchOutputConfig` to `send_command`
```
CloudWatchOutputConfig={
    'CloudWatchOutputEnabled': True
}
```

Example:
```
ssm_client.send_command(
    InstanceIds=[instance_id],
    DocumentName="AWS-RunShellScript",
    OutputS3BucketName=bucket,
    OutputS3KeyPrefix=folder,
    CloudWatchOutputConfig={
        'CloudWatchOutputEnabled': True
    },
    Parameters={
        'commands':[
            "cd /demo",
            "./bin/neo_agent_binary -a /tmp/sagemaker_edge_agent_example.sock -c neo_config.json" 
        ]
    }
)
```

Running log can be found in cloud watch log group `/aws/ssm/AWS-RunShellScript`